In [1]:
import imp
import pandas as pd

# Generate overall summary table - 2017

Table 3 of the 2015 report summarises all monitored and modelled loads for the whole of Norway, and for the four OSPAR regions. This notebook reads the output from the processing of the observed data (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_working_2016-17.ipynb)) and the modelling using NOPE (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/loads_unmonitored_regions_2016.ipynb)) and generates a Word version of Table 3. The code is based on the exploratory work [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb).

In [2]:
# Import custom RID functions
rid_func_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
                 r'\Python\rid\notebooks\useful_rid_code.py')

rid = imp.load_source('useful_rid_code', rid_func_path)

# Import model
nope_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
             r'\Python\rid\notebooks\nope.py')
nope = imp.load_source('nope', nope_path)

## 1. Summarise monitoring data

In [3]:
# Read station data
in_xlsx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Data\RID_Sites_List.xlsx')
stn_df = pd.read_excel(in_xlsx, sheet_name='RID_All')

# Get just cols of interest and drop duplicates 
# (some sites are in the same regine)
stn_df = stn_df.drop_duplicates(subset=['ospar_region', 'nve_vassdrag_nr'])

# Get catch IDs with calib data
calib_nds = set(stn_df['nve_vassdrag_nr'].values)

# Build network
in_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\NOPE_Annual_Inputs\nope_input_data_1990.csv')
g, nd_list = nope.build_calib_network(in_path, calib_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df['nve_vassdrag_nr'].isin(ds_nds)]

**Note:** Remember to update the year below.

In [4]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Loads_CSVs\loads_and_flows_all_sites_2017.csv')
mon_df = pd.read_csv(in_csv)

# Get just the downstream catchments
#mon_df = mon_df[mon_df['station_id'].isin(stn_df['station_id'].values)]

# Group by OSPAR region
mon_df1 = mon_df.groupby(['ospar_region', 'old_rid_group']).sum()

# Totals for Norway
mon_df2 = mon_df.groupby('old_rid_group').sum().reset_index()
mon_df2['ospar_region'] = 'NORWAY'
mon_df2.set_index(['ospar_region', 'old_rid_group'], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split('_')[1] != 'Est']
mon_df = mon_df[cols]
del mon_df['station_id']

# Rename cols to match template
mon_df['Flow rate_1000m3/day'] = mon_df['mean_q_1000m3/day']
del mon_df['mean_q_1000m3/day']

# Units are correct, so remove
mon_df.columns = [i.split('_')[0] for i in mon_df.columns]

mon_df.round(0)

Ag    As   Cd    Cr    Cu    Hg  NH4-N  \
ospar_region        old_rid_group                                            
LOFOTEN-BARENTS SEA rid_108        0.0   1.0  0.0   1.0   7.0   7.0   94.0   
                    rid_11         0.0   0.0  0.0   1.0   2.0   3.0   12.0   
                    rid_36         0.0   2.0  0.0   5.0  27.0  10.0  225.0   
NORTH SEA           rid_108        0.0   2.0  0.0   1.0  11.0  13.0  193.0   
                    rid_11         0.0   0.0  0.0   0.0   1.0   0.0   19.0   
                    rid_36         0.0   2.0  0.0   3.0  12.0  21.0  160.0   
NORWEGIAN SEA2      rid_108        0.0   1.0  0.0   1.0   7.0  10.0  156.0   
                    rid_11         0.0   1.0  0.0   1.0  13.0   0.0    9.0   
                    rid_36         0.0   3.0  0.0   8.0  25.0  20.0  205.0   
SKAGERAK            rid_108        0.0   1.0  0.0   1.0   3.0   4.0   81.0   
                    rid_11         0.0   8.0  1.0   9.0  50.0  48.0  587.0   
                    rid_36         0.0   2.0  0.0   1.0   7.0  11.0  126.0   
NORWAY              rid_108        0.0   5.0  0.0   4.0  29.0  34.0  524.0   
                    rid_11         0.0  10.0  1.0  11.0  66.0  52.0  627.0   
                    rid_36         0.0   9.0  1.0  16.0  71.0  62.0  716.0   

                                     NO3-N    Ni  PO4-P    Pb       SPM  \
ospar_region        old_rid_group                                         
LOFOTEN-BARENTS SEA rid_108          226.0   6.0   15.0   0.0   11851.0   
                    rid_11           195.0   1.0   15.0   0.0    6846.0   
                    rid_36           532.0  56.0  113.0   1.0   61467.0   
NORTH SEA           rid_108         3598.0   8.0   29.0   4.0   27531.0   
                    rid_11           431.0   1.0   13.0   0.0    4524.0   
                    rid_36          4216.0   4.0   54.0   4.0   42262.0   
NORWEGIAN SEA2      rid_108         1065.0   6.0   25.0   1.0   31321.0   
                    rid_11           465.0   3.0    6.0   0.0    7703.0   
                    rid_36          2722.0  21.0   76.0   1.0   99528.0   
SKAGERAK            rid_108         1437.0   2.0   16.0   1.0   13411.0   
                    rid_11         13519.0  28.0  314.0  10.0  263964.0   
                    rid_36          1399.0   3.0   17.0   3.0   16014.0   
NORWAY              rid_108         6326.0  21.0   85.0   6.0   84114.0   
                    rid_11         14610.0  33.0  348.0  10.0  283037.0   
                    rid_36          8869.0  84.0  260.0  10.0  219271.0   

                                       SiO2       TOC     TOTN   TOTP     Zn  \
ospar_region        old_rid_group                                              
LOFOTEN-BARENTS SEA rid_108         28725.0   30846.0   1322.0   25.0    8.0   
                    rid_11          16367.0   12030.0    705.0   26.0    1.0   
                    rid_36          92473.0   71082.0   3967.0  214.0   27.0   
NORTH SEA           rid_108         31558.0   32615.0   5833.0  103.0   34.0   
                    rid_11           3989.0    5335.0    779.0   29.0    4.0   
                    rid_36          38036.0   48394.0   7600.0  144.0   57.0   
NORWEGIAN SEA2      rid_108         25375.0   27538.0   2495.0   79.0   12.0   
                    rid_11          10405.0   11981.0    949.0   19.0   21.0   
                    rid_36          72013.0   69705.0   7027.0  157.0   58.0   
SKAGERAK            rid_108          9773.0   16711.0   2252.0   68.0   22.0   
                    rid_11         163721.0  208394.0  22897.0  571.0  124.0   
                    rid_36          21453.0   51078.0   3697.0   56.0   39.0   
NORWAY              rid_108         95431.0  107709.0  11902.0  275.0   75.0   
                    rid_11         194483.0  237740.0  25329.0  645.0  150.0   
                    rid_36         223976.0  240258.0  22291.0  572.0  181.0   

                                   Flow rate  
ospar_region        old_r

## 2. Data for unmonitored areas

**Note:** Remember to update the year below.

In [5]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Unmon_loads\unmon_loads_2017.csv')
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace('RENSEANLEGG', 'sew') for i in umon_df.columns]
umon_df.columns = [i.replace('INDUSTRI', 'ind') for i in umon_df.columns]
umon_df.columns = [i.replace('_tonn', '') for i in umon_df.columns]
umon_df.columns = [i.replace('AQUAKULTUR', 'fish') for i in umon_df.columns]

# Convert Hg to kgs
umon_df['sew_Hg'] = umon_df['sew_Hg']*1000
umon_df['ind_Hg'] = umon_df['ind_Hg']*1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,408822.0,13615.0,1128.0,2354.0,246.0,56691.0,9781.0,34781.0,711.0,677.0,...,1927.0,0.0,0.0,0.0,4.0,12.0,2.0,1.0,5.0,1088.0
LOFOTEN-BARENTS SEA,104554.0,1226.0,177.0,83.0,6.0,14141.0,2435.0,4980.0,102.0,106.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,271.0
NORTH SEA,151022.0,3714.0,440.0,412.0,90.0,19561.0,3370.0,14505.0,239.0,264.0,...,912.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,375.0
NORWEGIAN SEA2,143758.0,2828.0,392.0,984.0,109.0,22964.0,3971.0,12382.0,281.0,235.0,...,891.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,442.0
SKAGERAK,9488.0,5847.0,120.0,875.0,41.0,25.0,4.0,2915.0,88.0,72.0,...,4.0,0.0,0.0,0.0,2.0,8.0,1.0,0.0,4.0,0.0


## 3. Create table

In [6]:
# Create table 3
in_docx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Results\Word_Tables\2018Analysis_2017Data\Table_3_2017.docx')

rid.write_word_overall_table(mon_df, umon_df, in_docx)

Finished.
